****
## <center> <b> <span style="color:green;"> Python Proficiency for Scientific Computing and Data Science (PyPro-SCiDaS)  </span> </b></center>

### <center> <b> <span style="color:violet;">Project 3: Working Directory Generator </span> </b></center>

#### <center> <b> <span style="color:orange;"> 9 October 2025 </span> </b></center>

****

<h4 style="text-align:center;">Group 3 Members</h4>
<p style="text-align:center;">
Josiah Mutie <br>
Belise Kanziga <br>
Djadida Uwituze <br>
Patrick Nizeyimana <br>
Kutlo Gaone Kejang
</p>

****

### Project Objective

This project aims to automatically generate a directory structure for AIMS-Rwanda students using pure Python.

#### Inputs:
1. **Course List CSV** – one course name per line.
2. **Student List CSV** – each line in the format:
   `last_name, first_name`

#### Expected Outputs:
- A main working directory named: `AIMS-Rwanda-Workspace`.
- Inside it: one folder per student in the format: `Lastname, Firstname`).
- Inside each student folder: one subfolder per course.
- Inside each course folder: an empty `README.txt` file.
- Existing folders/files are **never overwritten**.

#### Additional Requirements:
- Validate CSVs exist and are not empty.
- Normalize names, that is, remove accents and special characters.
- Handle letter case for program to run regardless of lower/upper/mixed case.
- Add a simple main menu to allow re-running or quitting.

---

In [22]:
## Import only Python standard libraries (no external packages allowed)

import os
import csv
import unicodedata
import re
from typing import List

print(" ✅ Libraries imported successfully.")


 ✅ Libraries imported successfully.


In [23]:
# ================================================================================================
# STEP 1: Read text safely with encoding fallbacks
# ================================================================================================

def fallback_read(the_path: str) -> List[str]:
    """
    Reads all lines from a text file safely by trying multiple encodings 
    (UTF-8, CP1252, Latin-1, etc.).
    Input:
        the_path (str): Path to the text or CSV file to be read.
    Output:
        List[str]: A list containing all lines read from the file. 
                   Returns an empty list if reading fails.
    """
    encodings_to_try = ["utf-8-sig", "utf-8", "cp1252", "latin-1"]

    for encod in encodings_to_try:
        try:
            with open(the_path, encoding=encod, newline='') as file:
                lines = file.read().splitlines()
                print(f"✅ Successfully read '{the_path}' using {encod} encoding.")
                return lines
        except FileNotFoundError:
            print(f"⚠️ File not found: {the_path}")
            return []
        except Exception:
            continue  # silently try next encoding

    print(f"❌ Could not read file using any encoding: {the_path}")
    return []


In [24]:
# ================================================================================================
# STEP 2: Check and validate that the CSV file exists and is not empty
# ================================================================================================

def csv_non_empty(the_path: str) -> bool:
    """
    Checks whether a CSV file exists and is not empty.
    If missing or empty, prints a warning message instead of raising an error.
    Input: the_path (str) – path to the CSV file.
    Output: bool – True if file exists and has content, False otherwise.
    """
    try:
        if not os.path.isfile(the_path):
            print(f"⚠️ File not found: {the_path}")
            return False
        if os.path.getsize(the_path) == 0:
            print(f"⚠️ File is empty: {the_path}")
            return False
        print(f"✅ File is valid: {the_path}")
        return True
    except Exception as excep:
        print(f"❌ Error checking file '{the_path}': {excep}")
        return False


In [25]:
# ================================================================================================
# STEP 3: Clean and normalize student and course names
# ================================================================================================

def change_to_ascii(asc: str) -> str:
    """
    Converts special characters to plain ASCII.
    Example: 'García-López' → 'Garcia-Lopez'
    Input: asc (str) – text to normalize.
    Output: str – plain ASCII text.
    """
    if asc is None:
        return ""
    return unicodedata.normalize("NFKD", asc).encode("ascii", "ignore").decode("ascii")


def clean_std(p: str) -> str:
    if not p:
        return ""
    p = p.strip()
    p = change_to_ascii(p)
    p = p.replace("'", "").replace("`", "")
    p = re.sub(r'[^A-Za-z0-9 \-]', '', p)
    p = re.sub(r'\s+', ' ', p).strip()

    def hyphen(word):
        return "-".join([x.capitalize() for x in word.split("-")])

    return " ".join([hyphen(x) for x in p.split(" ")])


def clean_course(nam: str) -> str:
    """
    Cleans and formats course names, keeping important characters like dots and parentheses.
    Input: nam (str) – course name.
    Output: str – cleaned course name.
    """
    if not nam:
        return ""
    nam = nam.strip()
    nam = change_to_ascii(nam)
    nam = re.sub(r'[^A-Za-z0-9 \-\(\)\.\,_]', '', nam)
    nam = re.sub(r'\s+', ' ', nam).strip()
    return nam


In [26]:
# ================================================================================================
# STEP 4A: Read course lists with header handling
# ================================================================================================

def reading_courses(the_path: str, debug: bool = False) -> List[str]:
    """
    This function reads and cleans the list of courses from the course CSV file.
    Automatically skips headers such as 'Course', 'Course Name', etc.
    Input: Course List.csv
    Output: List[str]. A list of cleaned course names, ready for folder creation.
    """
    # ✅ Fixed: use the actual function name from STEP 2
    if not csv_non_empty(the_path):
        print(f"Skipping invalid or missing file: {the_path}")
        return []

    # Read the lines safely using fallback encoding
    lines = fallback_read(the_path)
    courses = []

    # Process and clean each line
    for r in lines:
        cleaned = clean_course(r)
        if not cleaned:
            continue

        # Skip header-like entries
        if cleaned.lower() in ["course", "course name", "courses", "name"]:
            if debug:
                print(f"Skipping header line: '{cleaned}'")
            continue

        courses.append(cleaned)

    if debug:
        print(f"✅ Found {len(courses)} course(s): {courses}")
    return courses


In [27]:
# ================================================================================================
# STEP 4B: Read and clean student list with header handling
# ================================================================================================

def reading_students(the_path: str, debug: bool = False) -> List[tuple]:
    """
    This function reads and cleans the list of students from a CSV file, skipping headers and invalid rows.
    Input: Student List.csv
    Output: A list of tuples (last, first) representing cleaned and standardized student names.
    """
    # ✅ Fixed: use csv_non_empty() instead of validate_csv()
    if not csv_non_empty(the_path):
        print(f"Skipping invalid or missing file: {the_path}")
        return []

    try:
        lines = fallback_read(the_path)
    except Exception as excep:
        print(f"Could not read file: {the_path}")
        if debug:
            print(f"[DEBUG] {excep}")
        return []

    students = []
    for r in lines:
        parts = [p.strip() for p in r.split(",")]
        if len(parts) < 2:
            continue

        # ✅ Fixed: use the correct cleaner name clean_std() instead of clean_student_part()
        last = clean_std(parts[0])
        first = clean_std(parts[1])

        # Skip header-like names
        if (last.lower(), first.lower()) in [
            ("lastname", "firstname"),
            ("surname", "name"),
            ("familyname", "givenname"),
        ]:
            if debug:
                print(f"Skipping header line: '{r}'")
            continue

        if last and first:
            students.append((last, first))

    if debug:
        print(f"✅ Found {len(students)} student(s): {students}")
    return students


In [28]:
# ================================================================================================
# STEP 5: Directory and file creation helper functions
# ================================================================================================

def safe_makedir(the_path: str) -> bool:
    """
    This function safely creates a directory if it doesn't already exist.
    Input: the_path (str) – path of the directory to be created.
    Output: bool – True if created, False if it already existed or an error occurred.
    """
    try:
        if os.path.isdir(the_path):
            # Folder already exists
            return False
        os.makedirs(the_path, exist_ok=True)
        return True
    except Exception as excep:
        print(f"❌ Error creating directory '{the_path}': {excep}")
        return False


def create_readme_if_missing(the_path: str) -> bool:
    """
    This function creates an empty README.txt file inside a given directory if it doesn't already exist.
    Input: the_path (str) – path to the directory (e.g., a student's course folder).
    Output: bool – True if README.txt was created, False if it already existed or an error occurred.
    """
    try:
        readme_path = os.path.join(the_path, "README.txt")
        if os.path.exists(readme_path):
            return False  # already there
        with open(readme_path, "w", encoding="utf-8") as file:
            file.write("")  # create empty README
        return True
    except Exception as excep:
        print(f"❌ Error creating README in '{the_path}': {excep}")
        return False


def make_student_folder_name(last: str, first: str) -> str:
    """
    Constructs a student's folder name in the format: 'Last, First'
    Input: last (str), first (str) – student's last and first names.
    Output: str – formatted folder name ('Last, First').
    """
    last_clean = last.replace(",", "").strip()
    first_clean = first.replace(",", "").strip()
    return f"{last_clean}, {first_clean}"


In [29]:
# ================================================================================================
# STEP 6: Core function to generate the workspace
# ================================================================================================

def make_workspace(student_csv: str, course_csv: str, root: str = "AIMS-Rwanda-Workspace") -> dict:
    """
    This function creates the complete AIMS-Rwanda workspace structure, including:
    - A main folder
    - Each student's personal folder
    - Course subfolders within each student folder
    - README.txt files inside each course folder

    Input:
        student_csv (str): Path to the student list CSV file (LastName, FirstName format)
        course_csv  (str): Path to the course list CSV file
        root        (str): Name of the main workspace directory (default: "AIMS-Rwanda-Workspace")

    Output:
        dict: A summary dictionary showing counts of folders and files created or skipped.
    """

    summary = {
        "students_processed": 0,
        "students_created": 0,
        "students_skipped": 0,
        "course_folders_created": 0,
        "course_folders_skipped": 0,
        "readmes_created": 0,
        "readmes_skipped": 0,
    }

    # Read input CSVs safely
    courses = reading_courses(course_csv)
    students = reading_students(student_csv)

    if not courses:
        print("⚠️ No valid courses found in the course CSV. Workspace generation skipped.")
        return summary

    if not students:
        print("⚠️ No valid students found in the student CSV. Workspace generation skipped.")
        return summary

    # Create the main workspace directory
    if safe_makedir(root):
        print(f"📁 Created main workspace directory: {root}")
    else:
        print(f"🗂️ Workspace already exists, adding new items to: {root}")

    # Loop through each student and create folders
    for last, first in students:
        summary["students_processed"] += 1
        student_folder = make_student_folder_name(last, first)
        student_path = os.path.join(root, student_folder)

        # Create the student folder
        if safe_makedir(student_path):
            print(f"👤 Created student folder: {student_folder}")
            summary["students_created"] += 1
        else:
            print(f"⏭️ Skipped existing student folder: {student_folder}")
            summary["students_skipped"] += 1

        # Create subfolders for each course
        for course in courses:
            course_path = os.path.join(student_path, course)

            if safe_makedir(course_path):
                print(f"  📚 Created course folder: {course}")
                summary["course_folders_created"] += 1
            else:
                summary["course_folders_skipped"] += 1

            # Add README.txt inside each course folder
            if create_readme_if_missing(course_path):
                summary["readmes_created"] += 1
            else:
                summary["readmes_skipped"] += 1

    print("\n✅ Workspace generation complete!")
    print(f"Summary: {summary}")
    return summary


In [30]:
# ================================================================================================
# STEP 7: Run the program automatically using default CSVs
# ================================================================================================

if __name__ == "__main__":
    student_csv = "Student List.csv"
    course_csv = "Course List.csv"

    print("\n🚀 Starting AIMS-Rwanda Workspace Generator...")
    summary = make_workspace(student_csv, course_csv)

    print("\n📊 Summary of workspace creation:")
    for key, value in summary.items():
        print(f"  {key}: {value}")


🚀 Starting AIMS-Rwanda Workspace Generator...
✅ File is valid: Course List.csv
✅ Successfully read 'Course List.csv' using utf-8-sig encoding.
✅ File is valid: Student List.csv
✅ Successfully read 'Student List.csv' using cp1252 encoding.
📁 Created main workspace directory: AIMS-Rwanda-Workspace
👤 Created student folder: Mutie, Josiah
  📚 Created course folder: Introduction to Python Programming
  📚 Created course folder: Physical Problem Solving
  📚 Created course folder: Mathematical Problem Solving
  📚 Created course folder: ICL, R, LaTex
  📚 Created course folder: Statistical Regression
👤 Created student folder: Kanziga, Belise
  📚 Created course folder: Introduction to Python Programming
  📚 Created course folder: Physical Problem Solving
  📚 Created course folder: Mathematical Problem Solving
  📚 Created course folder: ICL, R, LaTex
  📚 Created course folder: Statistical Regression
👤 Created student folder: Nizeyimana, Patrick
  📚 Created course folder: Introduction to Python Pr

In [31]:
def menu():
    """
    This function provides an interactive menu for generating AIMS-Rwanda workspaces.
    Input: User selections made through command-line prompts (e.g., file paths, confirmation choices).
    Output: Runs the workspace creation process based on user inputs and displays a summary in the console.
    """

    while True:
        print("\n========= 🧩 AIMS-Rwanda Workspace Generator =========")
        print("1. Generate new workspace")
        print("2. Quit")

        selection = input("Enter your choice (1 or 2): ").strip()

        if selection == "1":
            # Ask for file names (with defaults)
            student_csv = input("Enter student CSV path [default: Student List.csv]: ").strip() or "Student List.csv"
            course_csv = input("Enter course CSV path [default: Course List.csv]: ").strip() or "Course List.csv"
            root = input("Enter root folder name [default: AIMS-Rwanda-Workspace]: ").strip() or "AIMS-Rwanda-Workspace"

            try:
                # Check if the input CSV files exist
                if not os.path.exists(student_csv):
                    print(f"⚠️ Error: The student file '{student_csv}' was not found.")
                    print("Returning to main menu...")
                    continue  # safely go back to menu

                if not os.path.exists(course_csv):
                    print(f"⚠️ Error: The course file '{course_csv}' was not found.")
                    print("Returning to main menu...")
                    continue  # safely go back to menu

                # Run workspace creation safely
                summary = make_workspace(student_csv, course_csv, root)

                print("\n✅ Workspace generation completed successfully!")
                print("📊 Summary of workspace creation:")
                for key, value in summary.items():
                    print(f"  {key}: {value}")

            except Exception as e:
                print(f"❌ Unexpected error occurred: {e}")
                print("Returning to main menu...")
                continue  # do not crash; return to menu

        elif selection == "2":
            print("👋 Exiting the generator. Goodbye!")
            break

        else:
            print("⚠️ Invalid choice. Please enter 1 or 2.")

menu()


========= 🧩 AIMS-Rwanda Workspace Generator =========
1. Generate new workspace
2. Quit


Enter your choice (1 or 2):  1
Enter student CSV path [default: Student List.csv]:  k
Enter course CSV path [default: Course List.csv]:  j
Enter root folder name [default: AIMS-Rwanda-Workspace]:  s


⚠️ Error: The student file 'k' was not found.
Returning to main menu...

========= 🧩 AIMS-Rwanda Workspace Generator =========
1. Generate new workspace
2. Quit


Enter your choice (1 or 2):  1
Enter student CSV path [default: Student List.csv]:  Student List
Enter course CSV path [default: Course List.csv]:  Course List
Enter root folder name [default: AIMS-Rwanda-Workspace]:  AIMS Senegal Workspace


⚠️ Error: The student file 'Student List' was not found.
Returning to main menu...

========= 🧩 AIMS-Rwanda Workspace Generator =========
1. Generate new workspace
2. Quit


Enter your choice (1 or 2):  1
Enter student CSV path [default: Student List.csv]:  Student List.csv
Enter course CSV path [default: Course List.csv]:  Course List.csv
Enter root folder name [default: AIMS-Rwanda-Workspace]:  AIMS Senegal


✅ File is valid: Course List.csv
✅ Successfully read 'Course List.csv' using utf-8-sig encoding.
✅ File is valid: Student List.csv
✅ Successfully read 'Student List.csv' using cp1252 encoding.
📁 Created main workspace directory: AIMS Senegal
👤 Created student folder: Mutie, Josiah
  📚 Created course folder: Introduction to Python Programming
  📚 Created course folder: Physical Problem Solving
  📚 Created course folder: Mathematical Problem Solving
  📚 Created course folder: ICL, R, LaTex
  📚 Created course folder: Statistical Regression
👤 Created student folder: Kanziga, Belise
  📚 Created course folder: Introduction to Python Programming
  📚 Created course folder: Physical Problem Solving
  📚 Created course folder: Mathematical Problem Solving
  📚 Created course folder: ICL, R, LaTex
  📚 Created course folder: Statistical Regression
👤 Created student folder: Nizeyimana, Patrick
  📚 Created course folder: Introduction to Python Programming
  📚 Created course folder: Physical Problem So

Enter your choice (1 or 2):  2


👋 Exiting the generator. Goodbye!
